In [1]:
import pandas as pd

## LOAD NAIVE RESULTS

In [2]:
naive_results_dall_e_path_g_8='../data/test/discrete/naive/dall_e/naive_big_g_8.json'
naive_results_dall_e_path_g_32='../data/test/discrete/naive/dall_e/naive_big_g_32.json'
naive_results_dall_e_path_g_64='../data/test/discrete/naive/dall_e/naive_big_g_64.json'
naive_results_imagen_path_g_8='../data/test/discrete/naive/imagen/naive_medium_safe_big_g_8.json'
naive_results_imagen_path_g_32='../data/test/discrete/naive/imagen/naive_medium_safe_big_g_32.json'
naive_results_imagen_path_g_64='../data/test/discrete/naive/imagen/naive_medium_safe_big_g_64.json'

results_paths_dalle=[naive_results_dall_e_path_g_8,naive_results_dall_e_path_g_32,naive_results_dall_e_path_g_64]
results_paths_imagen=[naive_results_imagen_path_g_8,naive_results_imagen_path_g_32,naive_results_imagen_path_g_64]
result_dfs_dalle=[]
result_dfs_imagen=[]
for i in range(len(results_paths_dalle)):

    pd_results=pd.read_json(results_paths_dalle[i],orient='records')
    result_dfs_dalle.append(pd_results)
    pd_results=pd.read_json(results_paths_imagen[i],orient='records')
    result_dfs_imagen.append(pd_results)
    

## EVALUATE REJECTION AND SIMILARITY

In [3]:
results = []
seq_len=[8,32,64]
for i in range(len(result_dfs_dalle)):
    df=result_dfs_dalle[i]
    similarity_vals=df[~df['image_filename'].isna()]['similarity']
    df_none_image = df[df['image_filename'].isna()]
    blocked_text_true_pct = (df_none_image['blocked_text'].sum() / len(df_none_image)) * 100
    
    results.append({
        'model': 'Dall-e',
        'seq_len': seq_len[i],
        'rejection_perc': round((100 * len(df_none_image) / len(df)), 2),
        'rejection_text_perc': round(blocked_text_true_pct, 2),
        'rejection_image_perc': round(100 - blocked_text_true_pct, 2),
        'similarity': similarity_vals.mean()
    })
    
    df=result_dfs_imagen[i]
    similarity_vals=df[~df['image_filename'].isna()]['similarity']
    df_none_image = df[df['image_filename'].isna()]
    blocked_text_true_pct = (df_none_image['blocked_text'].sum() / len(df_none_image)) * 100
    
    results.append({
        'model': 'Imagen',
        'seq_len': seq_len[i],
        'rejection_perc': round((100 * len(df_none_image) / len(df)), 2),
        'rejection_text_perc': round(blocked_text_true_pct, 2),
        'rejection_image_perc': round(100 - blocked_text_true_pct, 2),
        'similarity': similarity_vals.mean()
    })
    
results= pd.DataFrame(results)
results=results.sort_values(by=['model'])
results

,model,seq_len,rejection_perc,rejection_text_perc,rejection_image_perc,similarity
0,Dall-e,8,19.39,100.00,0.00,0.233422
2,Dall-e,32,43.88,97.67,2.33,0.232998
4,Dall-e,64,44.90,86.36,13.64,0.198122
1,Imagen,8,86.73,96.47,3.53,0.403103
3,Imagen,32,86.32,96.34,3.66,0.409817
5,Imagen,64,88.17,95.12,4.88,0.398394
